In [ ]:
from keras import backend as K
from keras.applications import imagenet_utils
from keras.applications import xception as network 
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
from keras.models import Model
%matplotlib inline 

import numpy as np
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

inputShape = (299, 299, 3)

print("[INFO] loading {}...".format(network.__name__))
model = network.Xception(weights="imagenet")

# Generate GAP model to return both classification and final feature maps layers
GAP_model = Model(inputs=model.input,
                  outputs=(model.layers[-3].output, model.layers[-1].output))

In [ ]:
# This is the image to be classified with above network
image_file = '/tmp/workspace/Pictures/elephants.jpg'

print("[INFO] loading and pre-processing image...")

raw_image = load_img(image_file)
plt.imshow(raw_image)
plt.show()

image = load_img(image_file, target_size=inputShape)
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
image = network.preprocess_input(image)

print("[INFO] classifying image {} with {}...".format(image_file, network.__name__))
last_conv_output, preds = GAP_model.predict(image)
P = imagenet_utils.decode_predictions(preds)

results=[{'rank': i+1, 'score': prob*100.0, 'class': np.where(preds == prob)[1][0], 'id': imagenetID, 'label': label}
         for (i, (imagenetID, label, prob)) in enumerate(P[0])]

print('-' * 60)
print("{0:>5} |{1:>6}  |{2:>6} | {3:<12}| {4:<32}".format("Rank", "Score", "Class", "ImageNet ID", "Label"))
print('-' * 60)
for r in results:
    print("{rank:>5} |{score:>6.2f}% |{class:>6} | {id:<12}| {label:<48}".format(**r))
print('-' * 60)

In [ ]:
maxclass = np.argmax(preds[0])

# Perform the GAP-CAM process
last_conv_output = np.squeeze(last_conv_output)
all_amp_layer_weights = model.layers[-1].get_weights()[0]
amp_layer_weights = all_amp_layer_weights[:, maxclass]

fh, fw, fn = last_conv_output.shape
final_output = np.dot(last_conv_output.reshape((fh*fw, fn)), amp_layer_weights).reshape(fh,fw)
final_output /= np.max(final_output)
final_output = np.maximum(final_output, 0.0)

plt.figure(figsize=(15,15))
plt.subplot(1, 2, 1)
plt.imshow(final_output)
plt.subplot(1, 2, 2)
plt.imshow(raw_image)
plt.show()